In [11]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
from hmmlearn import hmm
import dask

# Setup example data

Define states and emission probabilities

In [2]:
state_means = np.array([0, 0.5, 1])
stdev = 0.1

Define a transition matrix

In [3]:
def generate_transition_matrix(n_states):
    transition_matrix = np.random.random(size=(n_states,n_states)) * 0.05
    for i in range(n_states):
        transition_matrix[i,i] = 1.0 - np.sum(transition_matrix[i,:]) + np.sum(transition_matrix[i,i])
    return transition_matrix

In [4]:
state_stds = np.array([stdev] * len(state_means))
transition_matrix = generate_transition_matrix(len(state_means))

Function to convert an underlying state into an emission (ie a state into an observed FRET value)

In [5]:
def get_emission_value(state):
    return np.random.normal(loc=state_means[state], scale=state_stds[state])

Write a generator function to generate a sequence of states and emissions

In [6]:
def generate_trace(n_steps, state_means, state_stds, transition_matrix):
    states = [0]
    values = [get_emission_value(states[-1])]
    shift = np.random.random() * .05 - .025
    for i in range(0,n_steps-1):
        current = states[-1]
        new_state = np.random.choice(np.arange(0,len(state_means)), p=transition_matrix[current])
        states.append(new_state)
        values.append(shift + get_emission_value(new_state))
    return states, values

In [7]:
len_trace = 500
num_traces = 80

states = []
values = []
for i in range(0,num_traces):
    states_sub, values_sub = generate_trace(len_trace, state_means, state_stds, transition_matrix)
    states.append(states_sub)
    values.append(values_sub)

In [8]:
px.line(y=[np.array(values)[0,:], state_means[np.array(states)[0,:]]])

# Functions for doing predictions

This is the standard pipeline, however I have found that it gives local minima very easily, and so it is best to run it multiple times and take the best result.  Note that it provides an initial guess of a uniform distribution for the initial state probabilities.

In [12]:
def do_fit(values, guessed_components):
    guessed_means = (np.arange(0,guessed_components)/(guessed_components-1)).reshape(-1,1)
    np_values = np.array(values).reshape(-1,1)

    model = hmm.GaussianHMM(n_components = guessed_components, covariance_type = "full", n_iter = 100, init_params="mcs",  means_prior=guessed_means,
                            )
    model.fit(np_values)
    # Predict the hidden states corresponding to observed X.
    Z = model.predict(np_values)
    score = model.score(np.array(values).reshape(-1,1))
    return model, Z, score

This will run the fitter multiple times and use the best.

In [13]:
def find_best_fit(values, guessed_components, num_trials=30):
    best_model = None
    best_score = -np.inf
    best_Z = None

    for i in range(num_trials):
        try:
            model, Z, score = do_fit(values, guessed_components)
            if score > best_score:
                best_score = score
                best_model = model
                best_Z = Z
            
        except:
            pass
    print(best_score)
    return best_model, best_model.means_[best_Z].squeeze()


In [14]:
def dask_wrapper(values, guessed_components, num_trials=30):
    model, predicted = find_best_fit(values, guessed_components, num_trials)
    return predicted


In [15]:
predicteds = [dask.delayed(dask_wrapper)(vals, 3, 30) for vals in values]
    

In [16]:
predicteds = dask.compute(*predicteds)

Model is not converging.  Current: 272.79543242710594 is not greater than 272.7957142876345. Delta is -0.00028186052855971866
Model is not converging.  Current: 325.81449201093284 is not greater than 325.8145185038774. Delta is -2.6492944584788347e-05
Model is not converging.  Current: 287.56840034264565 is not greater than 287.56850215497997. Delta is -0.00010181233432149384
Model is not converging.  Current: 325.81449569666137 is not greater than 325.8145085847643. Delta is -1.288810295818621e-05
Model is not converging.  Current: 96.14926089692435 is not greater than 96.15613084093586. Delta is -0.00686994401151253
Model is not converging.  Current: 327.9608124416831 is not greater than 327.961055908673. Delta is -0.00024346698990029836
Model is not converging.  Current: 272.7954042038586 is not greater than 272.79568916059196. Delta is -0.0002849567333669256


324.40282990564424


Model is not converging.  Current: 327.96085246793956 is not greater than 327.96090195199156. Delta is -4.9484051999115763e-05
Model is not converging.  Current: 272.7953727892004 is not greater than 272.79547956117676. Delta is -0.00010677197633413016


325.8145021619454
311.17649281378124
272.7953538337735


Model is not converging.  Current: 287.56841272002566 is not greater than 287.5684655700188. Delta is -5.284999315335881e-05


...

307.6434634066245


Model is not converging.  Current: 323.5636026596821 is not greater than 323.56398354658836. Delta is -0.00038088690627091637


326.5824029897853
326.3433672910865
323.56356145508107
325.3736718755288
300.9906192662522
318.06649531934903


Model is not converging.  Current: 258.24929056015753 is not greater than 258.2501115221293. Delta is -0.0008209619717831629


292.33284868035884
258.2492307867274
310.34502695611167


In [17]:
def find_transitions(Z):
    transitions = []
    for i in range(0,len(Z)-1):
        #transitions.append([Z[i],Z[i+1]])
        #if Z[i]!=Z[i+1]:
        if np.abs(Z[i] - Z[i+1])>0.1:
            transitions.append([Z[i],Z[i+1]])
    return np.array(transitions)

In [18]:
transitions = []
for predicted in predicteds:
    transitions.extend(find_transitions(predicted))
transitions = np.array(transitions)

In [19]:
px.histogram(np.array(predicteds).flatten(), nbins=100)

In [20]:
tdf = pd.DataFrame(data={'source':transitions[:,0], 'sink':transitions[:,1]})
f = px.density_heatmap(tdf, x='source', y='sink', nbinsx=200, nbinsy=200, width=800, height=700)
f.write_html('heatmap_5000.html')
f